# This script takes the rawdataset.csv, users.txt, face_vars.csv and usernames.csv and cleans it in a myriad of ways. It outputs a cleandata.csv where it is all combined

In [1]:
#import packages
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import matplotlib.image as mplt
from IPython.display import display, clear_output


importing a list of the users

In [2]:
with open("data\\users.txt",'r') as file:
    users = file.read().splitlines() 
    #users = [post[:-1] for post in users]


reading in all the csv files from the subfolders, and appending them to a dataframe

In [3]:
dataset = pd.DataFrame(columns=['post_nr', 'date', 'text', 'likes', 'user', 'piclink', 'postlink'])

In [4]:
for user in users:
    path =  "posts\\"+ user +"\\" 
    csv_path = path + user +".csv"
    pic_path = path + user + "_piclinks.txt"
    post_path = path + user + "_postlinks.txt"

    
    df = pd.read_csv(csv_path, usecols = [1,2,3,4])
    
    #concat the username
    df['user'] = user
    
    with open(pic_path, 'r') as file:
        pic_links = file.read().splitlines()
    with open(post_path, 'r') as file:
        post_links = file.read().splitlines()
    
    df['piclink'] = pic_links
    df['postlink'] = post_links
    #append the dataset
    dataset = dataset.append(df)


Save the dataset

In [5]:
dataset = dataset.reset_index()

In [6]:
dataset.to_csv("rawdataset.csv")

In [7]:
dataset = pd.read_csv("rawdataset.csv", usecols=[2,3,4,5,6])

In [8]:
dataset

,post_nr,date,text,likes,user
0,0,3 DAYS AGO,Nunarput ♥️,71 likes,akimati
1,1,4 DAYS AGO,So proud of @kunofencker. 🏆🏆🏆 You won the nati...,71 likes,akimati
2,2,FEBRUARY 2,Many people ask me how I can work as a parliam...,100 likes,akimati
3,3,JANUARY 23,Relatable af,108 likes,akimati
4,4,"DECEMBER 22, 2020",So excited for the kids to get home to us. I e...,19 likes,akimati
...,...,...,...,...,...
11679,19,"MARCH 19, 2019",På vej til Ilulissat ❤️ Glæder mig. Håber på g...,24 likes,aaja_chemnitz
11680,20,"FEBRUARY 23, 2019",Så er vi klar til at heppe på @nina_k_j og Jul...,47 likes,aaja_chemnitz
11681,21,"FEBRUARY 14, 2019",Disse dage besøger jeg Tasiilaq. Det er så smu...,41 likes,aaja_chemnitz
11682,22,"DECEMBER 11, 2018",Veloverstået samtale salon med @emileperonard ...,45 likes,aaja_chemnitz


Clean up some of the data. I begin by making likes into an integer

In [9]:
like_col = dataset['likes']
likes = []

In [10]:
for txt in like_col:
    if str(txt) != "nan":
        num = str(txt).split()[0]
        num = num.replace(",","")
        num = num.replace(".","")
        likes.append(int(num))
    else:
        likes.append(np.nan)

In [11]:
dataset['likes'] = likes

In [12]:
dataset

,post_nr,date,text,likes,user
0,0,3 DAYS AGO,Nunarput ♥️,71.0,akimati
1,1,4 DAYS AGO,So proud of @kunofencker. 🏆🏆🏆 You won the nati...,71.0,akimati
2,2,FEBRUARY 2,Many people ask me how I can work as a parliam...,100.0,akimati
3,3,JANUARY 23,Relatable af,108.0,akimati
4,4,"DECEMBER 22, 2020",So excited for the kids to get home to us. I e...,19.0,akimati
...,...,...,...,...,...
11679,19,"MARCH 19, 2019",På vej til Ilulissat ❤️ Glæder mig. Håber på g...,24.0,aaja_chemnitz
11680,20,"FEBRUARY 23, 2019",Så er vi klar til at heppe på @nina_k_j og Jul...,47.0,aaja_chemnitz
11681,21,"FEBRUARY 14, 2019",Disse dage besøger jeg Tasiilaq. Det er så smu...,41.0,aaja_chemnitz
11682,22,"DECEMBER 11, 2018",Veloverstået samtale salon med @emileperonard ...,45.0,aaja_chemnitz


I then change the date
- at the very first i change the once with "4 days ago " etc into they apporpriate dates
- then i split the standard date formatsinto day, month year - 3 different cols 
- i then make a combined date col
- i then make this col into a datetime format

In [13]:
date_col = dataset['date']
dates = []
day = []
month = []
year = []


In [14]:
def mapping(i):
    if i == "JANUARY" or i == "JANUAR":
        return 1
    elif i == "FEBRUARY" or i == "FEBRUAR":
        return 2
    elif i == "MARCH" or i == "MARTS":
        return 3
    elif i == "APRIL":
        return 4
    elif i == "MAY"or i == "MAJ":
        return 5
    elif i == "JUNE"or i == "JUNI":
        return 6
    elif i == "JULY"or i == "JULI":
        return 7
    elif i == "AUGUST":
        return 8
    elif i == "SEPTEMBER":
        return 9
    elif i == "OCTOBER"or i == "OKTOBER":
        return 10
    elif i == "NOVEMBER":
        return 11
    elif i == "DECEMBER":
        return 12
    return i


In [15]:
list_of_month = ["JANUARY",
                 "JANUAR",
                 "FEBRUARY",
                 "FEBRUAR",
                 "MARCH",
                 "MARTS",
                 "APRIL",
                 "MAY",
                 "MAJ",
                 "JUNE",
                 "JUNI",
                 "JULY",
                 "JULI",
                 "AUGUST",
                 "SEPTEMBER",
                 "OCTOBER",
                 "OKTOBER",
                 "NOVEMBER",
                 "DECEMBER"]

In [16]:
for dt in date_col:
    #if it is not nan
    if str(dt) != "nan":
        #this works for all the normal dates, also the ones with out year. but not for the "x days ago"
        split_date = dt.split(maxsplit = 3)
        #print(split_date)
        
        #if the month is first (format -m-d-y)
        if split_date[0] in list_of_month:
            month.append(mapping(split_date[0]))
            day.append(split_date[1].replace(",",""))
            try:
                year.append(split_date[2])
            except:
                #if there is no year, it means that it is the current year
                year.append(2021)
            #append in y-m-d format
            dates.append(str(year[-1]) + "-" + str(month[-1]) + "-" + str(day[-1]))
        #If the format is d-m-y
        elif split_date[1] in list_of_month:
            month.append(mapping(split_date[1]))
            day.append(split_date[0].replace(".",""))
            try:
                year.append(split_date[2])
            except:
                #if there is no year, it means that it is the current year
                year.append(2021)
                
            #append in y-m-d format
            dates.append(str(year[-1]) + "-" + str(month[-1]) + "-" + str(day[-1]))
        else:
            #there are 2 formats Danish and English for "3 days ago" or "For 3 dage siden"
            #they are all subtracted from the 18-02-2021. This is not entirely correct 
            # as some post were scraped on the 15th, 16th and 17th. However the notes on which are lost on a broken harddisk
            # it does not matter for the results. as i examine the periode clode to 26th Feb 2020. 
            if dt.split()[0].isdigit():
                day.append(18 - int(dt.split()[0]))
            elif dt.split()[1].isdigit():
                day.append(18 - int(dt.split()[1]))
            #all in February 2021
            month.append(2)
            year.append(2021)
            
            dates.append(str(year[-1]) + "-" + str(month[-1]) + "-" + str(day[-1]))


    else:
        day.append(np.nan)
        month.append(np.nan)
        year.append(np.nan)
        dates.append(np.nan)

In [17]:
dataset['date_clean'] = dates

I then make the date into a datetime object and calculate the days till the 26th of August 2020, the date of the speech. 

In [18]:
linde_date = datetime.strptime("2020-08-26", "%Y-%m-%d")
linde_date_06 = datetime.strptime("2020-09-06", "%Y-%m-%d")

In [19]:
datetimes = []
datedif = []
datedif_06 = []
for dt in dataset['date_clean']:
    try:
        datetimes.append(datetime.strptime(dt, "%Y-%m-%d"))
        datedif.append((datetimes[-1] - linde_date).days)
        datedif_06.append((datetimes[-1] - linde_date_06).days)

    except: 
        try:
            datetimes.append(datetime.strptime(dt, "%Y-%m-%d"))
            datedif.append((datetimes[-1] - linde_date).days)
            datedif_06.append((datetimes[-1] - linde_date_06).days)

        except:
            datetimes.append(np.nan)
            datedif.append(np.nan)
            datedif_06.append(np.nan)


In [20]:
dataset['datetime'] = datetimes
dataset['days_till_linde'] = datedif
dataset['days_till_linde_06'] = datedif_06

In [21]:
#os.chdir("C:\\Users\\August\\OneDrive - Københavns Universitet\\Documents\\Uni\\Kandidat i Statskundskab\\Speciale\\kode")
#dataset = pd.read_csv("cleandata.csv",index_col="Unnamed: 0")

In [22]:
#path = "C:\\Users\\August\\OneDrive - Københavns Universitet\\Documents\\Uni\\Kandidat i Statskundskab\\Speciale\\kode\\posts"

In [23]:
#dataset["path"] = [path + "\\" + str(dataset['user'][i]) + "\\pic_" + str(dataset['post_nr'][i]) + ".jpeg" for i in range(len(dataset))]
dataset["path"] = ["posts\\" + str(dataset['user'][i]) + "\\pic_" + str(dataset['post_nr'][i]) + ".jpeg" for i in range(len(dataset))]

## read in the gender amount of people data 

In [24]:
gender_dataset = pd.read_csv("data\\face_vars.csv",index_col="Unnamed: 0")

In [25]:
dataset['n_people'] = gender_dataset['n_people']
dataset['n_women'] = gender_dataset['n_women']
dataset['n_men'] = gender_dataset['n_men']

Remove imagewith more than 4 people (5 or more)

In [26]:
#Remove observations from before the alloted time)
dateend = datetime(2021,2,4)
dstart = datetime(2020,2,26)
linde_date = datetime.strptime("2020-08-26", "%Y-%m-%d")
dataset = dataset[dataset['datetime'] > dstart]

In [27]:
dataset['date_clean'] = [datetime.strptime(str(d), "%Y-%m-%d") for d in dataset['date_clean']]


C:\Users\tnv664\AppData\Local\Temp/ipykernel_14592/2280790094.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['date_clean'] = [datetime.strptime(str(d), "%Y-%m-%d") for d in dataset['date_clean']]


In [28]:
len(dataset)

10416

In [29]:
print(sum(dataset['n_people'] > 4))
print(dataset['n_people'].max())
dataset = dataset[dataset['n_people'] < 5]
dataset.reset_index(inplace=True)

224
47.0


In [30]:
len(dataset)

10192

In [31]:
dataset.reset_index(inplace=True)


I make a share of women var

In [32]:
for i in range(len(dataset)):
    
    try:
        dataset.loc[i,'share_women'] = int(dataset.loc[i,'n_women']) / int(dataset.loc[i,'n_people'])
        dataset.loc[i,'share_women_no_none'] = int(dataset.loc[i,'n_women']) / int(dataset.loc[i,'n_people'])
    except ZeroDivisionError:
        dataset.loc[i,'share_women'] = 0
        dataset.loc[i,'share_women_no_none'] = np.nan

I then define if it is before or after linde 

In [33]:
before =dataset['datetime'] < linde_date
after = dataset['datetime'] >= linde_date

In [34]:
#published on the 11th of september
before_06 =dataset['datetime'] < linde_date + timedelta(11)
after_06 = dataset['datetime'] >= linde_date + timedelta(11)


In [35]:
dataset.loc[before,'treatment'] = 0
dataset.loc[after,'treatment'] = 1
dataset.loc[before_06,'treatment_06'] = 0
dataset.loc[after_06,'treatment_06'] = 1

I proceed to remove the observations without a date

In [36]:
dataset.isnull().sum()

level_0                   0
index                     0
post_nr                   0
date                      0
text                    164
likes                   966
user                      0
date_clean                0
datetime                  0
days_till_linde           0
days_till_linde_06        0
path                      0
n_people                  0
n_women                   0
n_men                     0
share_women               0
share_women_no_none    3941
treatment                 0
treatment_06              0
dtype: int64

In [37]:
dataset = dataset[~dataset['date'].isnull()]

In [38]:
dataset.isnull().sum()

level_0                   0
index                     0
post_nr                   0
date                      0
text                    164
likes                   966
user                      0
date_clean                0
datetime                  0
days_till_linde           0
days_till_linde_06        0
path                      0
n_people                  0
n_women                   0
n_men                     0
share_women               0
share_women_no_none    3941
treatment                 0
treatment_06              0
dtype: int64

In [39]:
len(dataset)

10192

In [40]:
user_data = pd.read_csv("data\\usernames.csv", sep=";", encoding = "ISO-8859-1")

In [41]:
user_data

,user,navn,køn,parti,blok,dato for indsamling,valgperiode
0,akimati,Aki-Matilda Høegh-Dam,f,Siumut,r,11-03-2021,2019
1,alexvanopslagh,Alex Vanopslagh,m,Liberal Alliance,b,11-03-2021,2019
2,andreas.steenberg,Andreas Steenberg,m,Radikale venstre,r,11-03-2021,2019
3,anehalsboe,Ane Halsboe-Jørgensen,f,Socialdemokratiet,r,11-03-2021,2019
4,annehonoreoestergaard,Anne Honoré Østergaard,f,Venstre,b,11-03-2021,2019
...,...,...,...,...,...,...,...
131,troelsravn1,Troels Ravn,m,Socialdemokratiet,r,11-03-2021,2019
132,ullatornaes,Ulla Tørnæs,f,Venstre,b,11-03-2021,2019
133,victoria.r.velasquez,Victoria Vela?squez,f,Enhedslisten,r,11-03-2021,2019
134,zenia.stampe,Zenia Stampe,f,Radikale venstre,r,11-03-2021,2019


In [42]:
for i in range(len(user_data)):
    name = user_data.loc[i,'user']
    
    dataset.loc[dataset['user']==name,'name'] =  user_data.loc[i,'navn']
    dataset.loc[dataset['user']==name,'sex'] =  user_data.loc[i,'køn ']
    dataset.loc[dataset['user']==name, 'party'] =  user_data.loc[i,'parti']
    dataset.loc[dataset['user']==name, 'bloc'] =  user_data.loc[i,'blok']
    


add weeks and month years for plots

In [43]:
dataset = dataset.reset_index(drop = True)

In [44]:
for i in range(len(dataset)):
    if dataset.loc[i,'datetime'].year == 2020:
        dataset.loc[i,'week'] = int(dataset.loc[i,'datetime'].isocalendar()[1])
    
    
    elif dataset.loc[i,'datetime'].year == 2021:
        dataset.loc[i,'week'] = int(dataset.loc[i,'datetime'].isocalendar()[1] + 52)
    else:
         dataset.loc[i,'week'] = "old don't use"

monthyears = [datetime.strptime(str(dataset.loc[i,'datetime'].month) + "-" + str(dataset.loc[i,'datetime'].year),"%m-%Y") for i in range(len(dataset))]
dataset['monthyear'] = monthyears

Make top 10, superuser and percentiles

In [45]:
user_list = dataset.groupby(["name","bloc"]).agg({"name": ['count']})['name']
user_list = user_list.sort_values(by = "count", ascending=False)
user_list.reset_index(inplace=True)

In [47]:
q3, q1 = np.percentile(user_list['count'], [75 ,25])
iqr = q3 - q1
upper = user_list['count'].mean() + 1.5 * iqr
lower = user_list['count'].mean() - 1.5 * iqr
user_list['count'].mean()
print("upper bound:", upper)
print("lower bound:", lower)
superusers = user_list[user_list['count'] > upper]['name']
dataset['superuser'] = [1 if dataset.loc[i,'name'] in list(superusers) else 0 for i in range(len(dataset))]
ten_percent = user_list.head(13)['name']
dataset['tenpercent'] = [1 if dataset.loc[i,'name'] in list(ten_percent) else 0 for i in range(len(dataset))]
percentiles = list(np.percentile(user_list['count'], [0,10,20,30,40,50,60,70,80,90]))
#percentile = 1
for i in percentiles: 
    names = user_list[user_list['count'] >= i]['name']
    dataset.loc[dataset['name'].isin(names),'percentile'] = i


upper bound: 206.75
lower bound: -47.5


In [49]:
dataset[dataset['user'] != "nikolajvillumsen"]
dataset.loc[dataset['party'] == "Frie Grønne", 'party'] = "Løsgænger"
#frie grønne er løsgænger fra: https://www.ft.dk/da/medlemmer/mandatfordelingen

In [50]:
women_one = [1 if dataset.loc[i,'n_women'] > 0 else 0 for i in range(len(dataset))]
dataset['women_binary'] = women_one
men_one = [1 if dataset.loc[i,'n_men'] > 0 else 0 for i in range(len(dataset))]
dataset['men_binary'] = men_one

In [51]:
dataset['year_week'] = [str(dataset.date_clean[i].year) + "_" +  str(dataset.date_clean[i].week) for i in range(len(dataset))]

Standardize the amount of women

In [52]:
def standardize(x,mean,std):
    z = (x-mean) / std
    return z

In [53]:

for pol in dataset.user.unique():
    dat = dataset.loc[dataset['user'] == pol,'n_women']

    
    my = np.mean(dat)
    std = np.std(dat)
    dataset.loc[dataset['user'] == pol,'std_n_women'] = [standardize(dat[i],my,std) for i in dat.index]



C:\Users\tnv664\AppData\Local\Temp/ipykernel_14592/3642598273.py:2: RuntimeWarning: invalid value encountered in double_scalars
  z = (x-mean) / std


In [54]:
dataset.loc[dataset.std_n_women.isna(), 'std_n_women'] = 0

In [55]:
#dataset.days_till_linde  = dataset.days_till_linde * -1 #reverse
dataset.days_till_linde  =  dataset['days_till_linde'].astype(int) 

dataset['week'] = np.floor((dataset.days_till_linde) /7) #1 is the first week with treatment
dataset['fourteen_days'] = np.floor((dataset.days_till_linde) /14) #1 is the first week with treatment
dataset['month'] = np.floor((dataset.days_till_linde) /30) #1 is the first week with treatment

dataset['week'] = dataset['week'].astype(int)
dataset['fourteen_days'] = dataset['fourteen_days'].astype(int)
dataset['month'] = dataset['month'].astype(int)


dataset['week_after'] = [dataset.loc[i,'week'] if dataset.loc[i,'week'] >= 0 else -1 for i in range(len(dataset))]
dataset['fourteen_days_after'] = [dataset.loc[i,'fourteen_days'] if dataset.loc[i,'fourteen_days'] >= 0 else -1 for i in range(len(dataset))]
dataset['month_after'] = [dataset.loc[i,'month'] if dataset.loc[i,'month'] >= 0 else -1 for i in range(len(dataset))]
dataset['days_till_linde_after'] = [dataset.loc[i,'days_till_linde'] if dataset.loc[i,'days_till_linde'] >= 1 else -1 for i in range(len(dataset))]


#assign all period before as "control"

#recode the last week into the second to last, as there are only two obs in the last week
dataset.loc[dataset['week'] == 25,'week'] = 24
dataset.loc[dataset['week_after'] == 25,'week_after'] = 24

dataset.loc[dataset['month'] == 6,'month'] = 5 # very few observations in the last month
dataset.loc[dataset['month_after'] == 6,'month_after'] = 5

I then save the dataset. 

In [56]:
dataset.to_csv("data/cleandata.csv")